## Analýza zamestnancov VŠ

In [1]:
import pandas as pd

from pandasql import sqldf
# from sklearn import datasets

In [2]:
df = pd.read_csv('../data/preprocessed/regzam.csv', index_col=0)
display(df)

,Employee_ID,GivenName,FamilyName,DegreePedagogicalAcademic,DegreeMaster,DegreeDoctor,Employment_Workplace_University,Employment_Type,Employment_Subtype,Employment_BeginDate,Employment_Ratio,Guarantee_StudyProgram,Guarantee_University,Guarantee_Faculty,Guarantee_FormOfStudy,Guarantee_LevelOfStudy,Employment_Workplace_Faculty,Employment_EndDate
0,18799121690,Štefan,Kočan,Doc.,JUDr.,PhD.,Vysoká škola bezpečnostného manažérstva v Koši...,Docent,s titulom docent a vysokoškolským vzdelaním tr...,2020-10-01,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,14002860472,Dušan,Korgo,DOC,JUDR,PHD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,17922733804,Ján,Buzalka,prof.,PhDr.,CSc.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,16290288493,Jozef,Stieranka,prof.,Ing.,PhD.,Akadémia Policajného zboru,Profesor,s titulom profesor a vysokoškolským vzdelaním ...,2013-11-01,100,"['bezpečnostnoprávna ochrana osôb a majetku', ...","['Akadémia Policajného zboru', 'Akadémia Polic...","[None, None, None, None, None, None, None]","['1', '2', '1', '2', '2', '1', '2']","['1', '1', '2', '2', '2', '3', '3']",NaN,NaN
4,21401303068,Ľuboš,Wäldl,doc.,JUDr.,PhD.,Akadémia Policajného zboru,Profesor,s titulom docent a vysokoškolským vzdelaním tr...,2008-09-01,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32725,12110964830,Joža,Spurný,doc.,JUDr.PhDr.,Ph.D.,Vysoká škola zdravotníctva a sociálnej práce s...,Docent,s titulom docent a vysokoškolským vzdelaním tr...,2022-09-30,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32726,13500390262,Marcela,Harčárová,NaN,Mgr.,NaN,Vysoká škola zdravotníctva a sociálnej práce s...,Asistent,s vysokoškolským vzdelaním druhého stupňa,2022-09-01,50,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32727,17108478442,Jana,Juhásová,NaN,Mgr.,NaN,Vysoká škola zdravotníctva a sociálnej práce s...,Asistent,s vysokoškolským vzdelaním druhého stupňa,2022-09-01,50,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32728,15917135755,Miriama,Kubušová,NaN,Mgr.,NaN,Vysoká škola zdravotníctva a sociálnej práce s...,Asistent,s vysokoškolským vzdelaním druhého stupňa,2022-09-01,50,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
id_set = set(df["Employee_ID"])
print(len(id_set))

32730


-> máme Dataset s 32730 riadkami, pričom žiadne 2 Employee_ID nie sú rovnaké

In [4]:
name_set = set(df["FamilyName"] + " " + df["GivenName"])
len(name_set)

31639

-> 31639 rôznych mien; zostáva zistiť, či pri duplikátnych menách sa jedná o tú istú osobu, alebo nie.
Tiež by nás mohlo zaujímať, ako ukladali osoby, ktorým sa zmenilo meno...

In [5]:
def find_name_duplicates(df):
    name_set = set()
    name_duplicates = set()
    duplicates_ids = set()
    for index, row in df.iterrows():
        name = row["FamilyName"] + " " + row["GivenName"]
        if name in name_set:
            name_duplicates.add(name)
        else:
            name_set.add(name)
    for index, row in df.iterrows():
        name = row["FamilyName"] + " " + row["GivenName"]
        if name in name_duplicates:
            duplicates_ids.add(row["Employee_ID"])
    return tuple(name_duplicates), tuple(duplicates_ids)

In [6]:
name_duplicates, ids = find_name_duplicates(df)

In [7]:
q = f"SELECT * FROM df WHERE Employee_ID IN {ids}"
duplicates_df = sqldf(q, globals()).sort_values(by=['FamilyName', 'GivenName'])
display(duplicates_df)

,Employee_ID,GivenName,FamilyName,DegreePedagogicalAcademic,DegreeMaster,DegreeDoctor,Employment_Workplace_University,Employment_Type,Employment_Subtype,Employment_BeginDate,Employment_Ratio,Guarantee_StudyProgram,Guarantee_University,Guarantee_Faculty,Guarantee_FormOfStudy,Guarantee_LevelOfStudy,Employment_Workplace_Faculty,Employment_EndDate
1327,49910822214,Katarína,Adamíková,None,Mgr.,None,None,None,None,None,None,None,None,None,None,None,None,None
1576,18913486096,Katarína,Adamíková,None,Mgr.,None,None,None,None,None,None,None,None,None,None,None,None,None
114,39501978150,František,Adamčík,prof.,Ing.,CSc.,Technická univerzita v Košiciach,Profesor,s titulom profesor a vysokoškolským vzdelaním ...,2005-02-01,100,"['prevádzka lietadiel', 'prevádzka lietadiel',...","['Technická univerzita v Košiciach', 'Technick...","['Letecká fakulta', 'Letecká fakulta', 'Leteck...","['1', '1', '2']","['2', '3', '3']",Letecká fakulta,None
1325,17240205086,František,Adamčík,None,Ing.,PhD.,None,None,None,None,None,None,None,None,None,None,None,None
133,51642863772,Jana,Antalová,None,MUDr.,PhD.,Univerzita Komenského v Bratislave,Odborný asistent,s vysokoškolským vzdelaním tretieho stupňa,2020-08-15,100,None,None,None,None,None,Lekárska fakulta,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1876,15552653387,Tomáš,Žilka,None,MUDr.,None,Slovenská zdravotnícka univerzita v Bratislave,Zamestnanec vykonávajúci činnosti VŠ učiteľa m...,s vysokoškolským vzdelaním druhého stupňa,2022-02-01,100,None,None,None,None,None,Lekárska fakulta,None
1107,10056354153,Viktória,Žilková,None,Mgr.,None,None,None,None,None,None,None,None,None,None,None,None,None
1994,14058627798,Viktória,Žilková,None,None,MSc.,Univerzita Komenského v Bratislave,Výskumný pracovník - výskumník,s vysokoškolským vzdelaním druhého stupňa,2022-06-15,50,None,None,None,None,None,Lekárska fakulta,None
530,14639882132,Anna,Žuffová,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [8]:
df.query("GivenName == 'Viktória' and FamilyName == 'Žilková'")

,Employee_ID,GivenName,FamilyName,DegreePedagogicalAcademic,DegreeMaster,DegreeDoctor,Employment_Workplace_University,Employment_Type,Employment_Subtype,Employment_BeginDate,Employment_Ratio,Guarantee_StudyProgram,Guarantee_University,Guarantee_Faculty,Guarantee_FormOfStudy,Guarantee_LevelOfStudy,Employment_Workplace_Faculty,Employment_EndDate
17361,10056354153,Viktória,Žilková,NaN,Mgr.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32149,14058627798,Viktória,Žilková,NaN,NaN,MSc.,Univerzita Komenského v Bratislave,Výskumný pracovník - výskumník,s vysokoškolským vzdelaním druhého stupňa,2022-06-15,50,NaN,NaN,NaN,NaN,NaN,Lekárska fakulta,NaN


... V tomto bode sme zistili, že naše stiahnuté xml zrejme obsahuje len dáta pre aktuálnych zamestnancov...

Pre minulých (zamestnancov) neobsahuje údaje o zamestnaní, predovšetkým nevieme z dát zistiť, v akom období a kde boli zamestnaní, hoci ich meno je v dátach.

In [9]:
df.query("Employment_EndDate.isna()")

,Employee_ID,GivenName,FamilyName,DegreePedagogicalAcademic,DegreeMaster,DegreeDoctor,Employment_Workplace_University,Employment_Type,Employment_Subtype,Employment_BeginDate,Employment_Ratio,Guarantee_StudyProgram,Guarantee_University,Guarantee_Faculty,Guarantee_FormOfStudy,Guarantee_LevelOfStudy,Employment_Workplace_Faculty,Employment_EndDate
0,18799121690,Štefan,Kočan,Doc.,JUDr.,PhD.,Vysoká škola bezpečnostného manažérstva v Koši...,Docent,s titulom docent a vysokoškolským vzdelaním tr...,2020-10-01,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,14002860472,Dušan,Korgo,DOC,JUDR,PHD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,17922733804,Ján,Buzalka,prof.,PhDr.,CSc.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,16290288493,Jozef,Stieranka,prof.,Ing.,PhD.,Akadémia Policajného zboru,Profesor,s titulom profesor a vysokoškolským vzdelaním ...,2013-11-01,100,"['bezpečnostnoprávna ochrana osôb a majetku', ...","['Akadémia Policajného zboru', 'Akadémia Polic...","[None, None, None, None, None, None, None]","['1', '2', '1', '2', '2', '1', '2']","['1', '1', '2', '2', '2', '3', '3']",NaN,NaN
4,21401303068,Ľuboš,Wäldl,doc.,JUDr.,PhD.,Akadémia Policajného zboru,Profesor,s titulom docent a vysokoškolským vzdelaním tr...,2008-09-01,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32725,12110964830,Joža,Spurný,doc.,JUDr.PhDr.,Ph.D.,Vysoká škola zdravotníctva a sociálnej práce s...,Docent,s titulom docent a vysokoškolským vzdelaním tr...,2022-09-30,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32726,13500390262,Marcela,Harčárová,NaN,Mgr.,NaN,Vysoká škola zdravotníctva a sociálnej práce s...,Asistent,s vysokoškolským vzdelaním druhého stupňa,2022-09-01,50,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32727,17108478442,Jana,Juhásová,NaN,Mgr.,NaN,Vysoká škola zdravotníctva a sociálnej práce s...,Asistent,s vysokoškolským vzdelaním druhého stupňa,2022-09-01,50,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32728,15917135755,Miriama,Kubušová,NaN,Mgr.,NaN,Vysoká škola zdravotníctva a sociálnej práce s...,Asistent,s vysokoškolským vzdelaním druhého stupňa,2022-09-01,50,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## -> Až 32434 riadkov nemá EndDate, čo aj potvrdzuje našu hypotézu, že máme zlé dáta :(  (iba súčasné úväzky)

## **- Pracujú zamestnanci aj na viacerých pracoviskách a vidno v tom nejaké trendy alebo zhluky?**

In [112]:
df_workplaces = pd.read_csv('../data/preprocessed/regzam_workplaces_current.csv', index_col=0)
display(df_workplaces)

,Employee_ID,FamilyName,GivenName,DegreePedagogicalAcademic,DegreeMaster,DegreeDoctor,Employment_Workplace_University,Employment_Type,Employment_Ratio
0,18799121690,Kočan,Štefan,Doc.,JUDr.,PhD.,Vysoká škola bezpečnostného manažérstva v Koši...,Docent,100.00
1,16290288493,Stieranka,Jozef,prof.,Ing.,PhD.,Akadémia Policajného zboru,Profesor,100.00
2,21401303068,Wäldl,Ľuboš,doc.,JUDr.,PhD.,Akadémia Policajného zboru,Profesor,100.00
3,69828748660,Kostrec,Matej,NaN,JUDr.,PhD.,Akadémia Policajného zboru,Odborný asistent,100.00
4,91221451321,Krajníková,Magdaléna,NaN,JUDr.,PhD.,Akadémia Policajného zboru,Odborný asistent,100.00
...,...,...,...,...,...,...,...,...,...
14326,17628642100,Peter,Fabo,NaN,RNDr.,PhD.,Žilinská univerzita v Žiline,Výskumný pracovník - výskumník,20.00
14327,17628642100,Peter,Fabo,NaN,RNDr.,PhD.,Trenčianska univerzita Alexandra Dubčeka v Tre...,Výskumný pracovník - výskumník,100.00
14328,28653015119,Marianna,Berinšterová,NaN,Mgr.,PhD.,Prešovská univerzita v Prešove,Odborný asistent,100.00
14329,28653015119,Marianna,Berinšterová,NaN,Mgr.,PhD.,Vysoká škola medzinárodného podnikania ISM Slo...,Zamestnanec vykonávajúci činnosti VŠ učiteľa m...,5.00


In [114]:
# kontrola - Matusov skolitel :D :
df_workplaces[df_workplaces['FamilyName'] == 'Výrost']

,Employee_ID,FamilyName,GivenName,DegreePedagogicalAcademic,DegreeMaster,DegreeDoctor,Employment_Workplace_University,Employment_Type,Employment_Ratio
346,53060727347,Výrost,Tomáš,doc.,Ing.,PhD.,Ekonomická univerzita v Bratislave,Docent,100.0
347,53060727347,Výrost,Tomáš,doc.,Ing.,PhD.,Technická univerzita v Košiciach,Docent,50.0
